In [89]:
# !rm -rf images

In [90]:
#!pip install tensorflow numpy pandas matplotlib

In [91]:
import warnings
warnings.filterwarnings("ignore")

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import os



#params

In [93]:
stock="AAPL"
start = '2024-01-03'
end='2024-07-03'
#downloaded chart time frame
timeframe="1d"
#number of connected timeframes to learn
window = 12

epochs = 2

In [94]:
data = yf.download(stock, start=start, end=end,interval=timeframe)
data['SMA'] = data['Close'].rolling(window=window).mean()
data['EMA'] = data['Close'].ewm(span=window, adjust=False).mean()

data.dropna(inplace=True)


[*********************100%%**********************]  1 of 1 completed


In [95]:
data.head(10)

,Open,High,Low,Close,Adj Close,Volume,SMA,EMA
Date,,,,,,,,
2024-01-19,189.330002,191.949997,188.820007,191.559998,191.056732,68741000,185.186666,185.967470
2024-01-22,192.300003,195.330002,192.259995,193.889999,193.380600,60133900,185.989999,187.186321
2024-01-23,195.020004,195.750000,193.830002,195.179993,194.667221,42355600,187.095832,188.416116
2024-01-24,195.419998,196.380005,194.339996,194.500000,193.988998,53631300,188.205832,189.352098
2024-01-25,195.220001,196.270004,193.110001,194.169998,193.659882,54822100,188.923332,190.093314
2024-01-26,194.270004,194.759995,191.940002,192.419998,191.914474,44594000,189.529999,190.451265
2024-01-29,192.009995,192.199997,189.580002,191.729996,191.226288,47145600,189.991665,190.647993
2024-01-30,190.940002,191.800003,187.470001,188.039993,187.545975,55859400,190.195831,190.246762
2024-01-31,187.039993,187.100006,184.350006,184.399994,183.915543,55467800,190.069164,189.347259


In [96]:
def create_images(data,):
    if not os.path.exists('images'):
        os.makedirs('images')
        os.makedirs('images/1')
        os.makedirs('images/0')
    for i in range(window, len(data)):
        plt.figure(figsize=(5, 5))
        plt.xticks([])
        plt.yticks([])
        plt.plot(data['Close'][i-window:i])
        plt.plot(data['SMA'][i-window:i])
        plt.plot(data['EMA'][i-window:i])
        if data['Close'][i]>data['Close'][i-1]:
            plt.savefig(f'images/1/img_{i}.png')
        # 0 is Bearish
        else:
            plt.savefig(f'images/0/img_{i}.png')
        plt.close()

create_images(data)

In [97]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = datagen.flow_from_directory(
    'images',
    target_size=(128, 128),
    batch_size=1,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    'images',
    target_size=(128, 128),
    batch_size=1,
    class_mode='binary',
    subset='validation'
)

Found 82 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [98]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')


In [99]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)


Epoch 1/2
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 1.8663 - val_loss: 0.2605
Epoch 2/2
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.2964 - val_loss: 0.2575
